In [ ]:
import sys
import os

# Add project root (one level above notebooks/) to sys.path
sys.path.append(os.path.abspath(".."))

In [ ]:
# Notebook: Test Supervisor Agent with Vision + Whisper + RAG integration

from src.agents.supervisor import SupervisorAgent
from src.interfaces.whisper_bot import transcribe_audio

# Initialize Supervisor (will auto-load or create knowledge base)
agent = SupervisorAgent()

print("\n" + "="*60)
print("OPTION 1: Direct Image Input")
print("="*60)
image_path = "../data/PlantVillage/Tomato_Early_blight/0012b9d2-2130-4a06-a834-b1f3af34f57e___RS_Erly.B 8389.JPG"
result = agent.analyze_crop(image_path)
print(f"\nImage Analysis Result:")
print(f"  Label: {result['label']}")
print(f"  Confidence: {result['confidence']:.2%}")
print(f"  Action: {result['action']}")

# Query knowledge base about the detected disease
if result['label'] != 'PlantVillage':  # If a disease was detected
    query = f"What is {result['label']} and how do I treat it?"
    print(f"\nQuerying knowledge base: '{query}'")
    knowledge = agent.query_knowledge(query, top_k=2)
    print(f"\nRetrieved Knowledge:")
    for i, res in enumerate(knowledge['results'], 1):
        print(f"\n  [{i}] (Score: {res['score']:.3f})")
        print(f"      {res['metadata']['text'][:200]}...")

print("\n" + "="*60)
print("OPTION 2: Voice Note Input + RAG Query")
print("="*60)
audio_path = "../data/audio_sample.wav"
text = transcribe_audio(audio_path)
print(f"\nTranscribed Voice Note: '{text}'")

# Process the voice query through RAG
if text.strip():  # If transcription is not empty
    voice_response = agent.process_voice_query(text)
    print(f"\nRAG Response:")
    for i, res in enumerate(voice_response['results'], 1):
        print(f"\n  [{i}] (Score: {res['score']:.3f})")
        print(f"      {res['metadata']['text'][:200]}...")

print("\n" + "="*60)
print("OPTION 3: Direct Text Query")
print("="*60)
query = "How do I prevent tomato diseases?"
print(f"\nQuery: '{query}'")
response = agent.query_knowledge(query, top_k=3)
print(f"\nTop {len(response['results'])} Results:")
for i, res in enumerate(response['results'], 1):
    print(f"\n  [{i}] (Score: {res['score']:.3f})")
    print(f"      {res['metadata']['text']}")